# Generate Documents - Svenska Kyrkan i Wien

In [ ]:
# import sys
# !python -m pip install pandas
# !python -m pip install openpyxl
# !python -m pip install -r requirements.txt

In [ ]:
# pd.set_option('display.max_rows', 100)

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
from openpyxl import Workbook, load_workbook
import shutil
from copy import copy

In [ ]:
from datetime import datetime

In [ ]:
BASE_DIR = Path.cwd()
print(BASE_DIR)

### Filnamn

In [ ]:
# Jul eller pask
# marketType = 'pask'
marketType = 'julmaten'
# marketType = 'julmarknad'

Här kan man ändra namnen på alla filer och tabellblad för data som läses in ock skrivs ut

In [ ]:
inputDirectory = 'input'
outputDirectory = 'output'
templateDirectory = 'templates'

# Inputs
fileNameProducts = 'wc-product-export.csv'
fileNameOrders = 'orders.xlsx'
fileNameTimeslots = 'tidsbokning.csv'
fileNamePackingCategories = 'placeringsnycklar.xlsx'
fileNameSchedule = 'schedule.xlsx'

# Outputs
fileNamePackList = 'packlista.xlsx'
fileNamePickUpList = 'hämtningslista.xlsx'
fileNameSemleList = 'semlelista.xlsx'
fileNameMasterList = 'masterlista.xlsx'
fileNameDiffs = 'diffs.xlsx'
fileNameAllData = 'allData.xlsx'
fileNameTimeslotsOutput = 'tidsbokning.xlsx'

# Templates
fileNamePickUpListTemplate = 'hämtlista_template.xlsx'
fileNameSemleListTemplate = 'semlelista_template.xlsx'
fileNamePackingCategoriesTemplate = 'placeringsnycklar_[marketType].xlsx'
fileNameScheduleTemplate = 'schedule_[marketType].xlsx'
fileNamePackListTemplate = 'packlista_template_[marketType].xlsx'


fileNamePackingCategoriesTemplate = fileNamePackingCategoriesTemplate.replace('[marketType]', marketType)
fileNameScheduleTemplate = fileNameScheduleTemplate.replace('[marketType]', marketType)
fileNamePackListTemplate = fileNamePackListTemplate.replace('[marketType]', marketType)

### Export Data

Exportera data från WordPress:
- https://julmarknad.svenskakyrkan.at/wp-admin/
- https://julmaten.svenskakyrkan.at/wp-admin
- https://paskmarknad.svenskakyrkan.at/wp-admin/

För att exporten ska ha rätt format måste språket i WordPress vara engelska. Detta kan man ställa in i sin användarprofil. Det handlar om det som kallas "Interface Language" till skillnad från "Site Language"

1. Exportera produkter
    - Gå till __Products -> All Products__
    - Tryck på __Export__
    - Lämna standardinställningar (exportera allt) och tryck på __Generate CSV__
    - Spara filen i den härför avsedda mappen under namnet (exakt): _wc-product-export.csv_
2. Exportera beställningar
    - Gå till __WooCommerce-> Export Order__
    - Lämna standardinställningar och tryck på __Export__
    - Spara filen i den härför avsedda mappen under namnet (exakt): _orders.xlsx_
3. Exportera tidsbokningar
    - Gå till __WP Time Slots Booking Form__
    - Tryck på __Booking Orders__
    - Tryck på __Export to CSV__
    - Spara filen i den härför avsedda mappen under namnet (exakt): _tidsbokning.csv_
4. Se till att filen med placeringsnycklar står till förfogande (_placeringsnycklar.xlsx_)
5. Se till att filen med hämtningstider står till förfogande (_schedule.xlsx_)

De tv¨senare filerna ligger på OneDrive:
- https://svkyrkan-my.sharepoint.com/personal/katharina_hieke_svenskakyrkan_se/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fkatharina%5Fhieke%5Fsvenskakyrkan%5Fse%2FDocuments%2Fjulmarknad22&ga=1

### Read and Format Data

**Products**

In [ ]:
# Julbasaren has Artikelnummer instead of SKU
dfProducts = pd.read_csv(BASE_DIR / inputDirectory / fileNameProducts, usecols=['SKU', 'Short description', 'Regular price'])

In [ ]:
dfProducts.columns = ['ArticleNumber', 'ProductDescription', 'RegularPrice']

**Orders**

In [ ]:
dfOrders = pd.read_excel(str(BASE_DIR / inputDirectory / fileNameOrders), 
                         usecols=['Order Number', 'Full Name (Billing)', 'Email (Billing)',
                                  'Order Total Amount (- Refund)', 'Order Date', 'Customer Note',
                                  'Phone (Billing)', 'SKU', 'Quantity (- Refund)', 'Product Name',
                                  'Item Cost (inc. tax)', 'Item #', 'Info', 'pa_packn_kategori'])

In [ ]:
dfOrders.columns= ['OrderNumber', 'FullName', 'Email',
                   'OrderTotalAmount', 'OrderDate', 'CustomerNote',
                   'PhoneOrder', 'ArticleNumber', 'Quantity', 'ProductName',
                   'ItemCost', 'ItemID', 'Info', 'PackingCategory']

In [ ]:
# convert emails to lower case
dfOrders['Email'] = dfOrders['Email'].str.lower()

In [ ]:
dfOrders['PackingCategory'] = dfOrders['PackingCategory'].transform(lambda x: str(x))

In [ ]:
dfOrders['PackingCategoryKey'] = dfOrders['PackingCategory'].str.lower()

**Timeslots**

In [ ]:
if marketType == 'pask':
    # Pask
    dfTimeslots = pd.read_csv(BASE_DIR / inputDirectory / fileNameTimeslots, index_col=False,
                             parse_dates=['Time', 'app_date_1', 'app_datetime_1'], dayfirst=True,
                             usecols=['Time', 'Email',
                                      'app_date_1', 'app_slot_1', 'app_datetime_1',
                                      'Telefon', 'Unnamed: 21'], encoding='latin_1')
else:
    # Jul
    dfTimeslots = pd.read_csv(BASE_DIR / inputDirectory / fileNameTimeslots, index_col=False,
                             parse_dates=['Time', 'app_date_1', 'Select Date and Time'], dayfirst=False,
                             usecols=['Time', 'Email',
                                      'app_date_1', 'app_slot_1', 'Select Date and Time',
                                      'Name', 'Telefon'], encoding='latin_1')

In [ ]:
# Note that Name and Telephone are shifted in the export file of the paskmarknad!
dfTimeslots.columns=['OrderTime', 'Email',
                     'PickUpDate', 'PickUpTime', 'PickUpDateTime', 
                     'FullNameTimeslot', 'Telephone']

In [ ]:
# change pick-up time from string to real time format
#dfTimeslots['PickUpTime'] = dfTimeslots['PickUpTime'].transform(lambda x: x.time())

dfTimeslots['PickUpTime'] = dfTimeslots['PickUpTime'].transform(
    lambda t: datetime.strptime(t, '%H:%M').time())



In [ ]:
# convert emails to lower case
dfTimeslots['Email'] = dfTimeslots['Email'].str.lower()

In [ ]:
# re-export timeslots for good formatting
dfTimeslots.to_excel(BASE_DIR / outputDirectory / fileNameTimeslotsOutput)

**Packing Categories**

In [ ]:
Path.unlink(BASE_DIR / inputDirectory / fileNamePackingCategories, missing_ok=True)
shutil.copyfile(BASE_DIR / templateDirectory / fileNamePackingCategoriesTemplate, 
                BASE_DIR / inputDirectory / fileNamePackingCategories)

In [ ]:
dfPackingCategories = pd.read_excel(str(BASE_DIR / inputDirectory / fileNamePackingCategories), 
                                    header=1, skiprows=None, usecols=['info till Packlistan', 'Packningskategori'])

In [ ]:
dfPackingCategories.columns = ['PickUpArea', 'PackingCategory']

In [ ]:
dfPackingCategories['PackingCategory'] = dfPackingCategories['PackingCategory'].transform(lambda x: str(x))

In [ ]:
dfPackingCategories['PackingCategoryKey'] = dfPackingCategories['PackingCategory'].str.lower()

In [ ]:
dfPackingCategories['PickUpArea'] = dfPackingCategories['PickUpArea'].str.replace('_', ' ')

In [ ]:
dfPackingCategories['PackingCategoryIndex'] = dfPackingCategories.index

**Schedule**

In [ ]:
Path.unlink(BASE_DIR / inputDirectory / fileNameSchedule, missing_ok=True)
shutil.copyfile(BASE_DIR / templateDirectory / fileNameScheduleTemplate, 
                BASE_DIR / inputDirectory / fileNameSchedule)

In [ ]:
dfSchedule = pd.read_excel(str(BASE_DIR / inputDirectory / fileNameSchedule))

Skapa en extra spalter med date och datetime från dag och tid

In [ ]:
dfSchedule['Date'] = dfSchedule['Dag'].transform(lambda x: x.date())

In [ ]:
dfSchedule['DateTime'] = [datetime.combine(d, t) for d, t in zip(dfSchedule['Dag'], dfSchedule['Tid'])]

### Diffs

In [ ]:
def alignCellWidthFor(sheet):
    dims = {}
    for row in sheet.rows:
        for cell in row:
            if cell.value:
                dims[cell.column_letter] = max((dims.get(cell.column_letter, 0), len(str(cell.value))))    
    for col, value in dims.items():
        sheet.column_dimensions[col].width = value * 1.23

In [ ]:
Path.unlink(BASE_DIR / outputDirectory / fileNameDiffs, missing_ok=True)

**Har gett order men har ej bokat tid**

In [ ]:
orderEmails = dfOrders['Email'].unique()

In [ ]:
timeSlotEmails = dfTimeslots['Email'].unique()

In [ ]:
diffEmails = np.setdiff1d(orderEmails, timeSlotEmails)
dfOrderButNoTimeBooked = dfOrders[dfOrders['Email'].isin(diffEmails)].drop_duplicates(subset=['OrderNumber'])
dfOrderButNoTimeBooked = dfOrderButNoTimeBooked[['OrderNumber', 'FullName', 'Email',
       'OrderTotalAmount', 'OrderDate', 'CustomerNote', 'PhoneOrder']]

In [ ]:
dfOrderButNoTimeBooked.to_excel(BASE_DIR / outputDirectory / fileNameDiffs, sheet_name='Beställning utan tidsbokning')

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNameDiffs)
sheet = workbook['Beställning utan tidsbokning']
alignCellWidthFor(sheet)
workbook.save(filename=BASE_DIR / outputDirectory / fileNameDiffs)

**Tidsbokning utan order**

In [ ]:
diffEmails = np.setdiff1d(timeSlotEmails, orderEmails)
dfOrderButNoTimeBooked = dfTimeslots[dfTimeslots['Email'].isin(diffEmails)]
dfOrderButNoTimeBooked = dfOrderButNoTimeBooked[['Email', 'PickUpDate', 'PickUpTime', 'FullNameTimeslot', 'Telephone']]

In [ ]:
with pd.ExcelWriter(BASE_DIR / outputDirectory / fileNameDiffs,
                    mode='a') as writer:  
    dfOrderButNoTimeBooked.to_excel(writer, sheet_name= 'Tidsbokning utan beställning')

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNameDiffs)
sheet = workbook['Tidsbokning utan beställning']
alignCellWidthFor(sheet)
workbook.save(filename=BASE_DIR / outputDirectory / fileNameDiffs)

**Dubbla tidsbokningar**

In [ ]:
emailCounts = dfTimeslots['Email'].value_counts()
dfMoreThanOneBookings = dfTimeslots[dfTimeslots['Email'].isin(emailCounts.index[emailCounts.gt(1)])]
dfMoreThanOneBookings = dfMoreThanOneBookings.sort_values(by=['Email', 'PickUpDate', 'PickUpTime'])
dfMoreThanOneBookings = dfMoreThanOneBookings[['Email', 'PickUpDate', 'PickUpTime', 'FullNameTimeslot', 'Telephone']]

In [ ]:
with pd.ExcelWriter(BASE_DIR / outputDirectory / fileNameDiffs,
                    mode='a') as writer:  
    dfMoreThanOneBookings.to_excel(writer, sheet_name='Dubbla tidsbokningar')

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNameDiffs)
sheet = workbook['Dubbla tidsbokningar']
alignCellWidthFor(sheet)
workbook.save(filename=BASE_DIR / outputDirectory / fileNameDiffs)

In [ ]:
# after reporting, we remove the duplicates
dfTimeslots.drop_duplicates(subset=['Email'], inplace=True)

**Artikel saknas i produktplaceringslista**

In [ ]:
orderSKUs = dfOrders['ArticleNumber'].unique()

In [ ]:
produktListaArtiklar = dfProducts['ArticleNumber'].unique()

In [ ]:
skuDiff = np.setdiff1d(orderSKUs, produktListaArtiklar)

In [ ]:
missingProducts = dfOrders[dfOrders['ArticleNumber'].isin(skuDiff)][['ArticleNumber', 'ProductName']]
missingProducts.drop_duplicates(subset=['ArticleNumber'], inplace=True)
missingProducts = missingProducts.sort_values(by='ArticleNumber')
print(len(missingProducts))

In [ ]:
with pd.ExcelWriter(BASE_DIR / outputDirectory / fileNameDiffs,
                    mode='a') as writer:  
    missingProducts.to_excel(writer, sheet_name='Saknas i produktlista')

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNameDiffs)
sheet = workbook['Saknas i produktlista']
alignCellWidthFor(sheet)
workbook.save(filename=BASE_DIR / outputDirectory / fileNameDiffs)

**Packningskategori saknas**

In [ ]:
orderPackingCategoriesList = dfOrders['PackingCategoryKey'].unique()

In [ ]:
packingCategoriesList = dfPackingCategories['PackingCategoryKey'].unique()

In [ ]:
packingCategoriesDiff = np.setdiff1d(orderPackingCategoriesList, packingCategoriesList)

In [ ]:
packingCategoriesDiff

In [ ]:
orderWithMissingCategory = dfOrders[dfOrders['PackingCategoryKey'].isin(packingCategoriesDiff)]
orderWithMissingCategory = orderWithMissingCategory[['PackingCategory', 'OrderNumber', 'ArticleNumber', 'ProductName', 'FullName', 'Email']]
orderWithMissingCategory = orderWithMissingCategory.sort_values(by='PackingCategory')
print(len(orderWithMissingCategory))

In [ ]:
with pd.ExcelWriter(BASE_DIR / outputDirectory / fileNameDiffs,
                    mode='a') as writer:  
    orderWithMissingCategory.to_excel(writer, sheet_name='Packningskategori saknas')

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNameDiffs)
sheet = workbook['Packningskategori saknas']
alignCellWidthFor(sheet)
workbook.save(filename=BASE_DIR / outputDirectory / fileNameDiffs)

### Merge and Select Data

In [ ]:
dfOrders.shape

In [ ]:
# Merge orders with timeslots
dfAllData = pd.merge(dfOrders, dfTimeslots, on='Email')

In [ ]:
dfAllData.shape

In [ ]:
# Merge with products
dfAllData = pd.merge(dfAllData, dfProducts, on='ArticleNumber', suffixes=['', None])

In [ ]:
dfAllData.shape

In [ ]:
# Merge with packing category
dfAllData = pd.merge(dfAllData, dfPackingCategories, on='PackingCategoryKey')

In [ ]:
dfAllData.shape

In [ ]:
dfAllData.to_excel(BASE_DIR / outputDirectory / fileNameAllData)

### Packlista

In [ ]:
def getFirstCellForNamedRange(workbook, sheet, range_name):
    cellNameList = []
    firstCellTuple = ()
    if range_name in workbook.defined_names:
        cellNameList = [(s, c) for s, c in workbook.defined_names[range_name].destinations if (s == sheet.title)]
    if cellNameList:
        firstCellTuple = cellNameList[0]
    return firstCellTuple[1]

In [ ]:
def fillCellForAttribute(rangeName, workbook, template, sheet, row_offset, value):
    templateCellNumber = getFirstCellForNamedRange(workbook, template, rangeName)
    targetCellNumber = template[templateCellNumber].offset(row=row_offset).coordinate
    sheet[targetCellNumber] = value

In [ ]:
def getMergedCellsOrNone(sheet, row, column):
    cell = sheet.cell(row, column)
    for mergedCell in sheet.merged_cells.ranges:
        if (cell.coordinate in mergedCell):
            return mergedCell
    return None

In [ ]:
def insertRowAndCopyFormat(template, sheet, source_row, target_row):
    sheet.insert_rows(idx=target_row)
    sheet.row_dimensions[target_row].height = template.row_dimensions[source_row].height
    for col in range(1, 11):
        sheet.cell(row=target_row, column=col)._style = template.cell(row=source_row, column=col)._style
        mergedCells = getMergedCellsOrNone(template, source_row, col)
        if mergedCells is not None and col == mergedCells.bounds[0]:
            mergedCellsCopy = copy(mergedCells)
            if target_row > source_row:
                mergedCellsCopy.shift(row_shift=target_row-source_row)
                sheet.merge_cells(range_string=(mergedCellsCopy.coord))

In [ ]:
Path.unlink(BASE_DIR / outputDirectory / fileNamePackList, missing_ok=True)
shutil.copyfile(BASE_DIR / templateDirectory / fileNamePackListTemplate, 
                BASE_DIR / outputDirectory / fileNamePackList)

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNamePackList)
template_sheet = workbook['template']

In [ ]:
dfAllData = dfAllData.sort_values(by=['PickUpDate', 'PickUpTime', 'PackingCategoryIndex'])

In [ ]:
dfGroupedByOrderNumber = dfAllData.groupby(by=['PickUpDate', 'PickUpTime', 'OrderNumber'])

In [ ]:
articleCell = getFirstCellForNamedRange(workbook, template_sheet, 'ArticleNumber')
headerRow = template_sheet[articleCell].row

for index, group in dfGroupedByOrderNumber:
    new_sheet = workbook.copy_worksheet(template_sheet)
    order_number = group.iloc[0]['OrderNumber']
    new_sheet.title = 'PO_' + str(order_number)
    for rangeName in ['FullName', 'OrderTotalAmount', 'CustomerNote', 'OrderNumber', 'Telephone', 'PickUpDate', 'PickUpTime']:
        fieldValue = group.iloc[0][rangeName]
        fillCellForAttribute(rangeName, workbook, template_sheet, new_sheet, 0, fieldValue)
    new_sheet.delete_rows(idx=headerRow, amount=2)
    # sort=False is important because packing categories should be according to order in file (not alphabetical)
    dfGroupedByPickUpArea = group.groupby('PickUpArea', sort=False)
    i = 0
    d = 0
    for area, subGroup in dfGroupedByPickUpArea:
        for sub_index, row in subGroup.iterrows():
            # sub_index is the (arbitrary) row number and cannot be used instead of i!
            insertRowAndCopyFormat(template_sheet, new_sheet, headerRow, headerRow+i)
            for rangeName in ['ArticleNumber', 'Quantity', 'ProductName', 'ItemCost', 'PickUpArea']:
                fieldValue = row[rangeName]
                fillCellForAttribute(rangeName, workbook, template_sheet, new_sheet, i, fieldValue)
            i+=1
            fillCellForAttribute('RowNumber', workbook, template_sheet, new_sheet, i-1, i-d)
        insertRowAndCopyFormat(template_sheet, new_sheet, headerRow+1, headerRow+i)
        i+=1
        d+=1
    for j in range(0,10):
        offset = headerRow+i
        new_sheet.row_dimensions[offset+j].height = template_sheet.row_dimensions[headerRow+2+j].height
    fillCellForAttribute('NumberOfRows', workbook, template_sheet, new_sheet, 0, str(i-d))

In [ ]:
workbook.remove(template_sheet)

In [ ]:
workbook.save(filename=BASE_DIR / outputDirectory / fileNamePackList)

### Hämtlista

In [ ]:
weekDayDict = {
0: 'Mån',
1: 'Tis',
2: 'Ons',
3: 'Tor',
4: 'Fre',
5: 'Lör',
6: 'Sön'}

In [ ]:
Path.unlink(BASE_DIR / outputDirectory / fileNamePickUpList, missing_ok=True)
shutil.copyfile(BASE_DIR / templateDirectory / fileNamePickUpListTemplate,
                BASE_DIR / outputDirectory / fileNamePickUpList)

In [ ]:
workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNamePickUpList)
template_sheet = workbook['template']

In [ ]:
dfGroupedByPickUpDate = dfAllData.drop_duplicates(subset=['OrderNumber']).groupby('PickUpDate')

In [ ]:
pickUpTimeCell = getFirstCellForNamedRange(workbook, template_sheet, 'PickUpTime')
orderNumberCol = template_sheet[pickUpTimeCell].column

for ts, group in dfGroupedByPickUpDate:
    new_sheet = workbook.copy_worksheet(template_sheet)
    new_sheet.title = 'Hämtas ' + str(ts.date())
    fillCellForAttribute('Weekday', workbook, template_sheet, new_sheet, 0, weekDayDict[ts.weekday()])
    fillCellForAttribute('PickUpDate', workbook, template_sheet, new_sheet, 0, ts.date())
    fillCellForAttribute('CreationDateTime', workbook, template_sheet, new_sheet, 0, str(datetime.now()))
    headerRow = template_sheet[pickUpTimeCell].row
    new_sheet.delete_rows(idx=headerRow, amount=2)
    group = group.sort_values(by=['FullName'])
    dfGroupedByPickUpTime = group.groupby('PickUpTime')
    scheduledTimes = dfSchedule[dfSchedule['Date']==ts.date()]['Tid']
    i = 0
    d = 0
    for time in scheduledTimes:
        insertRowAndCopyFormat(template_sheet, new_sheet, headerRow, headerRow+i)
        fillCellForAttribute('PickUpTime', workbook, template_sheet, new_sheet, i, time)
        i+=1
        d+=1
        if time in dfGroupedByPickUpTime.indices:
            subGroup = dfGroupedByPickUpTime.get_group(time)
            for index, row in subGroup.iterrows():
                insertRowAndCopyFormat(template_sheet, new_sheet, headerRow+1, headerRow+i)
                for rangeName in ['OrderNumber', 'FullName', 'Telephone', 'OrderTotalAmount', 'CustomerNote']:
                    fieldValue = row[rangeName]
                    fillCellForAttribute(rangeName, workbook, template_sheet, new_sheet, i-1, fieldValue)
                i+=1
                fillCellForAttribute('RowNumber', workbook, template_sheet, new_sheet, i-2, i-d)
        else:
            insertRowAndCopyFormat(template_sheet, new_sheet, headerRow+1, headerRow+i)
            i+=1
            fillCellForAttribute('RowNumber', workbook, template_sheet, new_sheet, i-2, i-d)

In [ ]:
workbook.remove(template_sheet)

In [ ]:
workbook.save(filename=BASE_DIR / outputDirectory / fileNamePickUpList)

### Semlelista

In [ ]:
if marketType == 'pask':
    Path.unlink(BASE_DIR / outputDirectory / fileNameSemleList, missing_ok=True)
    shutil.copyfile(BASE_DIR / templateDirectory / fileNameSemleListTemplate, 
                    BASE_DIR / outputDirectory / fileNameSemleList)

In [ ]:
if marketType == 'pask':
    workbook = load_workbook(filename=BASE_DIR / outputDirectory / fileNameSemleList)
    template_sheet = workbook['template']

In [ ]:
if marketType == 'pask':
    articleNumberSemla = '#H_11#'
    dfOrdersSemla = dfAllData[dfAllData['ArticleNumber'] == articleNumberSemla]
    dfOrdersSemla = dfOrdersSemla[['ArticleNumber', 'ProductName', 'Quantity', 'OrderNumber', 'FullName', 'Email', 'PickUpDate', 'PickUpTime']]
    dfOrdersSemla['PickUpDateTime'] = [datetime.combine(d, t) for d, t in zip(dfOrdersSemla['PickUpDate'], dfOrdersSemla['PickUpTime'])]
    if dfOrdersSemla.size > 0:
        dfOrdersSemla['PickUpHour']  = dfOrdersSemla['PickUpDateTime'].dt.hour

In [ ]:
if marketType == 'pask':
    dfGroupedByPickUpDate = dfOrdersSemla.groupby(['PickUpDate'])

In [ ]:
if marketType == 'pask':
    orderNumberCell = getFirstCellForNamedRange(workbook, template_sheet, 'OrderNumber')
    headerRow = template_sheet[orderNumberCell].row

    new_sheet = workbook.copy_worksheet(template_sheet)
    new_sheet.title = 'Semla-Lista'

    for rangeName in ['ArticleNumber', 'ProductName']:
        fieldValue = ''
        if dfOrdersSemla.size > 0:
            fieldValue = dfOrdersSemla.iloc[0][rangeName]
        fillCellForAttribute(rangeName, workbook, template_sheet, new_sheet, 0, fieldValue)
    new_sheet.delete_rows(idx=headerRow, amount=2)

    i = 0

    for day, group in dfGroupedByPickUpDate:
        dfGroupedByPickUpArea = group.groupby('PickUpHour')
        
        for hour, subGroup in dfGroupedByPickUpArea:
            for sub_index, row in subGroup.iterrows():
                insertRowAndCopyFormat(template_sheet, new_sheet, headerRow, headerRow+i)
                for rangeName in ['Quantity', 'OrderNumber', 'FullName', 'Email', 'PickUpDate', 'PickUpTime']:
                    fieldValue = row[rangeName]
                    fillCellForAttribute(rangeName, workbook, template_sheet, new_sheet, i, fieldValue)
                i+=1
            insertRowAndCopyFormat(template_sheet, new_sheet, headerRow+1, headerRow+i)
            fillCellForAttribute('Day', workbook, template_sheet, new_sheet, i-1, day.strftime('%d.%m'))
            timeIntervalString = str(hour) + ':00 - ' + str(hour+1) + ':00'
            fillCellForAttribute('Hour', workbook, template_sheet, new_sheet, i-1, timeIntervalString)
            fillCellForAttribute('NumberOfItemsName', workbook, template_sheet, new_sheet, i-1, 'Antal:')
            numberOfItemsString = subGroup['Quantity'].sum()
            fillCellForAttribute('NumberOfItems', workbook, template_sheet, new_sheet, i-1, numberOfItemsString)
            i+=2
            
        for j in range(0,10):
            offset = headerRow+i
            new_sheet.row_dimensions[offset+j].height = template_sheet.row_dimensions[headerRow+2+j].height

In [ ]:
if marketType == 'pask':
    workbook.remove(template_sheet)

In [ ]:
if marketType == 'pask':
    workbook.save(filename=BASE_DIR / outputDirectory / fileNameSemleList)

### Masterlista

In [ ]:
dfMasterList = dfAllData[['OrderNumber', 'FullName', 'Email', 'OrderTotalAmount', 'OrderDate', 
              'Telephone', 'CustomerNote', 'Info', 'PickUpDate', 'PickUpTime']]

In [ ]:
dfMasterList.columns = ['Order Number', 'Full Name (Billing)', 'Email (Billing)', 'Order Total Amount', 'Order Date', 
              'Telephone', 'Customer Note', 'Info ok', 'Hämtningsdag', 'Hämtning Tid']

In [ ]:
dfMasterList = dfMasterList.drop_duplicates()

In [ ]:
dfMasterList.to_excel(BASE_DIR / outputDirectory / fileNameMasterList)